In [1]:
import numpy as np
import cvxpy as cp
import mosek
import time
import scipy.stats
#from sklearn.model_selection import train_test_split

import phi_divergence as phi
#import robust_sampling as rs
import itertools as itertools
#import dataio
#import util

In [11]:
def make_center(lb,up,m):
    delta = (up-lb)/m
    center = np.arange(lb+delta/2,up,delta)
    return(center)          

def get_freq(cpt,data,lb,up):
    Freq = np.zeros(len(cpt))
    delta = (up-lb)/len(cpt)
    for i in range(len(data)):
        index = int(np.floor((data[i]-lb)/delta))
        Freq[index] = Freq[index] + 1
    return(Freq/len(data))

def solve_rc(Omega,a,b):
    d = len(a[0])
    x = cp.Variable(d, nonneg = True)
    z = cp.Variable(d)
    w = cp.Variable(d)
    constraints = [cp.norm(z,1)+Omega*cp.norm(w,2) + a[0] @ x <= b]
    for i in range(d):
        constraints.append(z[i] + w[i] == -a[i+1] @ x) 
    obj = cp.Maximize(cp.sum(x))
    prob = cp.Problem(obj,constraints)
    prob.solve(solver = cp.MOSEK)
    return(x.value)

def lower_bound(alpha,p,S,N):
    N_v = len(p)
    q = cp.Variable(N_v, nonneg = True)
    t = cp.Variable(N_v, nonneg = True)
    r = 1/N*scipy.stats.chi2.ppf(1-alpha, N_v-1)
    constraints = [cp.sum(q) == 1]
    f_obj = 0
    for i in range(N_v):
        if S[i] == 1:
            f_obj = f_obj + q[i]
        z = cp.vstack([q[i]-p[i],(t[i]-q[i])/2])
        constraints.append(cp.norm(z,2) <= (t[i]+q[i])/2)
    constraints.append(cp.sum(t) <= r)
    obj = cp.Minimize(f_obj)
    prob = cp.Problem(obj,constraints)
    prob.solve(solver = cp.MOSEK)
    return(prob.value, q.value)
    
def cpt_feas(cpt_arr,x,a,b,indices):
    d = len(cpt_arr)
    m = len(cpt_arr[0])
    S = np.zeros(len(indices))
    for i in range(len(S)):
        const = a[0]
        for j in range(d):
            const = const + cpt_arr[j][indices[i][j]] * a[j+1] 
        #print(const.dot(x))
        if const.dot(x) <= b:
            S[i] = 1
    return(S)
    
def Ishan(dim_x,alpha,beta,N,Omega,step_size,a,b):
    xi = np.random.uniform(size = (dim_x,N))*2-1
    cpt_arr = []
    lb = -1
    up = 1
    m = 10
    delta = 0
    indices = np.asarray(list((itertools.product(np.arange(m), repeat = dim_x))))
    p = np.zeros(len(indices))
    freq_ct = []
    for i in range(dim_x):
        cpt_arr.append(make_center(lb,up,m))
        freq_ct.append(get_freq(cpt_arr[i], xi[i],-1,1))
    #print(cpt_arr[0])
    #print(freq_ct)
    for j in range(len(indices)):
        p[j] = 1
        for k in range(dim_x):
            p[j] = p[j] * freq_ct[k][indices[j][k]]
    lowerbound = -np.inf
    while lowerbound < beta:
        x = solve_rc(Omega,a,b)
        #print(x)
        S = cpt_feas(cpt_arr,x,a,b,indices)
        lowerbound,q = lower_bound(alpha,p,S,N)
        print('Omega', Omega)
        print('lowerbound',lowerbound)
        print('objective', np.sum(x))
        print()
        Omega = Omega + step_size

In [10]:
a = [np.array([1,1]), np.array([1,0]), np.array([0,1])]
b = 10
Omega = 0.1
step_size = 0.01
alpha = 0.001
beta = 0.8                #### lower bound too low? S is mostly 1
dim_x = 2
N = 100 * 100
Ishan(dim_x,alpha,beta,N,Omega,step_size,a,b)  

Omega 0.1
lowerbound 0.4840585143338805
objective 9.339591174283981
Omega 0.11
lowerbound 0.4840585143338805
objective 9.278316354163948
Omega 0.12
lowerbound 0.4840585143338805
objective 9.217840312354479
Omega 0.13
lowerbound 0.4840585143338805
objective 9.15814753109521
Omega 0.14
lowerbound 0.4840585143338805
objective 9.099222890667363
Omega 0.15000000000000002
lowerbound 0.5753075934257946
objective 9.041051659388291
Omega 0.16000000000000003
lowerbound 0.5753075934257946
objective 8.983619479438332
Omega 0.17000000000000004
lowerbound 0.5753075934257946
objective 8.926912292517422
Omega 0.18000000000000005
lowerbound 0.5753075934257946
objective 8.870916615444948
Omega 0.19000000000000006
lowerbound 0.5753075934257946
objective 8.815619027502294
Omega 0.20000000000000007
lowerbound 0.5753075934257946
objective 8.761006564296217
Omega 0.21000000000000008
lowerbound 0.5753075934257946
objective 8.707066572819848
Omega 0.22000000000000008
lowerbound 0.5753075934257946
objective 8.6